In [1]:
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2
import time

In [2]:
# camera Module 0 for integrated webacm (PC camera)
# camera Module 1 for GoPro camera

In [2]:
cap = cv2.VideoCapture(2)
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
        print("maiu")
    else:
        cv2.imshow("final_result" , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(1)
time.sleep(10)
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
        print("Failed")
    else:
        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blured_image = cv2.GaussianBlur(gray_image, (15, 15), 0)
        edged = cv2.Canny(blured_image, 50, 100)
        edged = cv2.dilate(edged, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts[0]) > 0:
            cnts = imutils.grab_contours(cnts)
            (cnts, _) = contours.sort_contours(cnts)
            cnts = [x for x in cnts if cv2.contourArea(x) > 100]
            cv2.drawContours(frame, cnts, -1, (0,255,0), 3)
            if len(cnts) > 0:
                ref_object = cnts[0]
                box = cv2.minAreaRect(ref_object)
                box = cv2.boxPoints(box)
                box = np.array(box, dtype="int")
                box = perspective.order_points(box)
                (tl, tr, br, bl) = box
                dist_in_pixel = euclidean(tl, tr)
                dist_in_cm = 1.5
                pixel_per_cm = dist_in_pixel/dist_in_cm
                for cnt in cnts:
                    box = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(box)
                    box = np.array(box, dtype="int")
                    box = perspective.order_points(box)
                    (tl, tr, br, bl) = box
                    cv2.drawContours(frame, [box.astype("int")], -1, (0, 0, 255), 2)
                    mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2), tl[1] + int(abs(tr[1] - tl[1])/2))
                    mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2), tr[1] + int(abs(tr[1] - br[1])/2))
                    wid = euclidean(tl, tr)/pixel_per_cm
                    ht = euclidean(tr, br)/pixel_per_cm
                    cv2.putText(frame, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                    cv2.putText(frame, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                cv2.imshow("final_result" , frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
img_path = "example_02.jpg"

# Read image and preprocess
image = cv2.imread(img_path)
cv2.imshow("image_loaded" , image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("image_gray" , gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
blured_image = cv2.GaussianBlur(gray_image, (15, 15), 0) #gaussianBlur only admits odd numbers (as bigger the number, more blur added to the image)
cv2.imshow("image_blured" , blured_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
edged = cv2.Canny(blured_image, 50, 100)
cv2.imshow("image_edged" , edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
edged = cv2.dilate(edged, None, iterations=1)
cv2.imshow("image_edged_2" , edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
edged = cv2.erode(edged, None, iterations=1)
cv2.imshow("image_edged_3" , edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# Find contours
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [ ]:
# Sort contours from left to right as leftmost contour is reference object
(cnts, _) = contours.sort_contours(cnts)

In [ ]:
# Remove contours which are not large enough
cnts = [x for x in cnts if cv2.contourArea(x) > 100]

In [ ]:
cv2.drawContours(image, cnts, -1, (0,255,0), 3)

In [ ]:
# Reference object dimensions
# Here for reference I have used a 2cm x 2cm square
ref_object = cnts[0]
box = cv2.minAreaRect(ref_object)
box = cv2.boxPoints(box)
box = np.array(box, dtype="int")
box = perspective.order_points(box)
(tl, tr, br, bl) = box
dist_in_pixel = euclidean(tl, tr)
dist_in_cm = 2
pixel_per_cm = dist_in_pixel/dist_in_cm

In [ ]:
# Draw remaining contours
for cnt in cnts:
	box = cv2.minAreaRect(cnt)
	box = cv2.boxPoints(box)
	box = np.array(box, dtype="int")
	box = perspective.order_points(box)
	(tl, tr, br, bl) = box
	cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
	mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0])/2), tl[1] + int(abs(tr[1] - tl[1])/2))
	mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0])/2), tr[1] + int(abs(tr[1] - br[1])/2))
	wid = euclidean(tl, tr)/pixel_per_cm
	ht = euclidean(tr, br)/pixel_per_cm
	cv2.putText(image, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
	cv2.putText(image, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

In [ ]:
cv2.imshow("final_result" , image)
cv2.waitKey(0)
cv2.destroyAllWindows()